In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [4]:
df_coddi = pd.read_csv('coddi_simple_model.csv')

In [6]:
df_coddi.head()

,item_id,name,latitude,longitude,global_average_rating,reviews_count,plan,review_id,average_rating,timestamp,user_id
0,628858c72e93ec29e035fcd9,'Agua del Manzano',-33.002696,-71.062806,4.00,1,628858c72e93ec29e035fcec,4.00,2016-08-16 10:53:40,628858c72e93ec29e035fceb,NaN
1,628858c72e93ec29e035fced,'Aldea de Tulor',-22.964724,-68.237332,4.00,1,628858c72e93ec29e035fd00,4.00,2015-11-16 03:31:19,628858c72e93ec29e035fcff,NaN
2,628858c72e93ec29e035fd01,'Aguada Los Sapos',-26.165451,-70.630121,3.00,1,628858c72e93ec29e035fd14,3.00,2015-12-14 08:09:36,628858c72e93ec29e035fd13,NaN
3,628858c72e93ec29e035fd15,'Alerce Milenario - Lañilahual',-40.170024,-73.465583,4.50,1,628858c72e93ec29e035fd28,4.50,2015-06-18 02:41:54,628858c72e93ec29e035fd27,NaN
4,628858c72e93ec29e035fd29,'Arboretum',-33.725292,-70.487355,2.25,1,628858c72e93ec29e035fd3c,2.25,2015-08-18 10:30:57,628858c72e93ec29e035fd3b,NaN


In [8]:
df_coddi['name']

0                         'Agua del Manzano'
1                           'Aldea de Tulor'
2                         'Aguada Los Sapos'
3            'Alerce Milenario - Lañilahual'
4                                'Arboretum'
                        ...                 
8143        'Car wash up to midsize campers'
8144    'Summer traffic - plan accordingly!'
8145                     'Public WC Toilets'
8146            'Beautiful new tarmac road!'
8147                'Laguna espejo/glaciar '
Name: name, Length: 8148, dtype: object

In [19]:
#Vectorización en Inglés
tfidf = TfidfVectorizer(stop_words="english")
df_coddi['name'] = df_coddi['name'].fillna("")
tfidf_matrix = tfidf.fit_transform(df_coddi['name'])

In [20]:
#Creación del indicador de similaridad entre POI
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [21]:
#Creación de indices por POI
indices = pd.Series(df_coddi.index, index= df_coddi['name']).drop_duplicates()
indices

name
'Agua del Manzano'                         0
'Aldea de Tulor'                           1
'Aguada Los Sapos'                         2
'Alerce Milenario - Lañilahual'            3
'Arboretum'                                4
                                        ... 
'Car wash up to midsize campers'        8143
'Summer traffic - plan accordingly!'    8144
'Public WC Toilets'                     8145
'Beautiful new tarmac road!'            8146
'Laguna espejo/glaciar '                8147
Length: 8148, dtype: int64

In [27]:
#Verificación -> nombre con '###'
indices["'Car wash up to midsize campers'"]

8143

In [30]:
def get_recommendations(name, cosine_sim = cosine_sim):
    idx = indices[name]
    sim_scores = enumerate(cosine_sim[idx])
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    
    sim_index = [i[0] for i in sim_scores]
    print(df_coddi['name'].iloc[sim_index])

In [34]:
# Entrega lista de recomendación
get_recommendations("'Aguada Los Sapos'")

398         'Los Andes'
1983        'Los Andes'
6879       'Los Leones'
170     'Los Arrayanes'
799      'Los Pioneros'
2269       'los hornos'
445        'Los Olivos'
175      'Los Huemules'
329       'Los Perales'
1379       'los boldos'
Name: name, dtype: object


In [36]:
# Entrega lista de recomendación
get_recommendations("'Car wash up to midsize campers'")

4899                       'Estacionamento - car wash'
4755                               'Lubricam Car Wash'
8022                              ''El Che' Car Wash '
5600                                   'Andes Campers'
5617                                   'Andes Campers'
8078                           'Pre-shipping car wash'
8030                'Car Wash Polariauto - Great job!'
5716    'Car/bike wash with a small garrage Alejandro'
1821                                     'Car parking'
4257                                     'Car parking'
Name: name, dtype: object


In [37]:
# Entrega lista de recomendación
get_recommendations("'Arboretum'")

0                                 'Agua del Manzano'
1                                   'Aldea de Tulor'
2                                 'Aguada Los Sapos'
3                    'Alerce Milenario - Lañilahual'
5     'Alto del Naranjo vía San Carlos de Apoquindo'
6                                 'Baños Campanario'
7               'Alto del Naranjo por Puente Ñilhue'
8                                     'Baños Azules'
9                                     'Baños Coyuco'
10                             'Baños del Tupungato'
Name: name, dtype: object
